# Chapter 18. Concurrency with asyncio
> Concurrency is about dealing with lots of things at once.  
Parallelism is about doing lots of things at once.  
<br>
Rob Pike

In [5]:
# spinner thread
import threading
import itertools
import time


def spin(msg, done):
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        print(status, flush=True, end='\r')
        if done.wait(.1):
            break
    print(' ' * len(status), end='\r')


def slow_function():
    time.sleep(3)
    return 42


def supervisor():
    done = threading.Event()
    spinner = threading.Thread(target=spin, args=('thinking!', done))
    print('spinner object:', spinner)
    spinner.start()
    result = slow_function()
    done.set()
    spinner.join()
    return result


def main():
    result = supervisor()
    print('Answer:', result)


main()

spinner object: <Thread(Thread-8, initial)>
Answer: 42 


In [6]:
# spinner asyncio
import asyncio
import itertools


async def spin(msg):
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        print(status, flush=True, end='\r')
        try:
            await asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
    print(' ' * len(status), end='\r')


async def slow_function():
    await asyncio.sleep(3)
    return 42


async def supervisor():
    spinner = asyncio.create_task(spin('thinking!'))
    print('spinner object:', spinner)
    result = await slow_function()
    spinner.cancel()
    return result


def main():
    result = asyncio.run(supervisor())
    print('Answer:', result)


main()

RuntimeError: asyncio.run() cannot be called from a running event loop